In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import math
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementations import *
from pathlib import Path
import zipfile
import warnings

my_file = Path("../data/train.csv")
if not my_file.is_file():
    with zipfile.ZipFile('../data/train.csv.zip', 'r') as zip_ref:
        zip_ref.extractall('../data')

DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Exploratory data analysis and feature processing
As a first step, we obtain the number of events and features.

In [3]:
events = pd.read_csv(DATA_TRAIN_PATH)
n_events = events.shape[0]
n_features = events.shape[1] - 2
print('Number of events:', n_events)
print('Number of features:', n_features)
events.head()

Number of events: 250000
Number of features: 30


,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


We now obtain the percentage of events for each prediction.

In [4]:
events['Prediction'].value_counts()/n_events * 100

b    65.7332
s    34.2668
Name: Prediction, dtype: float64

Now we know that we only have two possible predictions (b or s). This is why we can think about this problem as a **Binary Classification** in which **Y** can take two values $Y \in {b, s}$ where b and s are the class labels.

In [5]:
events.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,224999.500000,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,...,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,72168.927986,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,...,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,100000.000000,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,...,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,162499.750000,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,...,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,224999.500000,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,...,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,287499.250000,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,...,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


### Dealing with missing values
We can now check the number of missing values per feature.

In [6]:
columns = []
for columnName in events.columns[2:]:
    column = events[columnName].to_numpy()
    n = column[np.where(column == -999.0)].shape[0]
    if n > 0:
        columns.append(columnName)
        print('{columnName} is missing {n} values.'.format(columnName=columnName, n=n))

DER_mass_MMC is missing 38114 values.
DER_deltaeta_jet_jet is missing 177457 values.
DER_mass_jet_jet is missing 177457 values.
DER_prodeta_jet_jet is missing 177457 values.
DER_lep_eta_centrality is missing 177457 values.
PRI_jet_leading_pt is missing 99913 values.
PRI_jet_leading_eta is missing 99913 values.
PRI_jet_leading_phi is missing 99913 values.
PRI_jet_subleading_pt is missing 177457 values.
PRI_jet_subleading_eta is missing 177457 values.
PRI_jet_subleading_phi is missing 177457 values.


According to the documentation, the value for the mass is -999.0 when the topology of the event was too far from the expected one. We can see that there are 38114 missing values for this feature (DER_mass_MMC). To replace this values, we are going to use the median of the rest of the values for the feature. Other option will be to use the mean, but we will stick with the first option as it is more robust when we have outliers.

In [7]:
def replace_na_values(data):
    """Replace NA values (-999.0) with the mean value of their column."""
    for i in range(data.shape[1]):
        msk = (data[:, i] != -999.)
        # Replace NA values with mean value
        median = np.median(data[msk, i])
        if math.isnan(median):
            median = 0
        data[~msk, i] = median
    return data

Regarding the other missing values, these depend on the number of jets of the event (PRI_jet_num):
- If it is 0, a specific set S of features presents missing values.
- If it is 1, only a specific subset $S' \subset S$ of the features presents missing values.
- If it is either 2 or 3, there are no missing values.  

In order to have this into account, we will create 3 different masks so we can create 3 different models that fit better.

In [8]:
def get_masks(x):
    """Returns 3 masks depending on the number of jets of the event."""
    # Note that 'PRI_jet_num' is the row 22
    event0 = (x[:, 22] == 0)
    event1 = (x[:, 22] == 1)
    event2 = (x[:, 22] != 0) & (x[:, 22] != 1)
    
    return [event0, event1, event2]

### Dealing with related features

We now check if there are any obvious relationships between the features.

In [9]:
corr_matrix = events.corr()
corr_matrix.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
Id,1.00,0.00,-0.01,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00
DER_mass_MMC,0.00,1.00,-0.46,0.17,0.20,0.16,0.16,0.16,0.23,0.05,0.20,-0.02,0.36,0.16,0.13,0.00,-0.01,0.10,0.01,-0.00,-0.23,0.01,0.22,0.22,0.25,0.25,0.25,0.16,0.16,0.16,0.19
DER_mass_transverse_met_lep,-0.01,-0.46,1.00,0.19,-0.25,-0.18,-0.19,-0.18,0.04,0.02,-0.15,0.35,-0.42,-0.18,-0.15,-0.00,0.00,0.31,-0.01,0.00,0.18,-0.02,-0.17,-0.21,-0.23,-0.22,-0.22,-0.18,-0.18,-0.18,-0.21
DER_mass_vis,-0.00,0.17,0.19,1.00,-0.06,-0.03,-0.04,-0.03,0.58,-0.00,0.09,0.10,-0.09,-0.03,0.29,0.00,-0.00,0.41,0.00,-0.00,-0.09,-0.00,0.05,-0.03,-0.02,-0.01,-0.01,-0.03,-0.03,-0.03,-0.05
DER_pt_h,0.00,0.20,-0.25,-0.06,1.00,0.52,0.53,0.52,-0.54,0.31,0.83,0.09,0.54,0.52,0.41,0.00,0.01,0.36,0.01,-0.00,0.68,0.01,0.78,0.62,0.62,0.56,0.56,0.53,0.52,0.52,0.81
DER_deltaeta_jet_jet,-0.00,0.16,-0.18,-0.03,0.52,1.00,0.95,1.00,-0.30,0.27,0.67,0.04,0.37,1.00,0.19,0.00,0.00,0.17,0.01,0.00,0.31,0.01,0.62,0.87,0.55,0.52,0.52,1.00,1.00,1.00,0.71
DER_mass_jet_jet,0.00,0.16,-0.19,-0.04,0.53,0.95,1.00,0.94,-0.30,0.25,0.68,0.03,0.37,0.95,0.20,0.00,0.00,0.16,0.01,-0.00,0.32,0.01,0.62,0.81,0.52,0.49,0.49,0.95,0.95,0.95,0.72
DER_prodeta_jet_jet,-0.00,0.16,-0.18,-0.03,0.52,1.00,0.94,1.00,-0.30,0.27,0.67,0.04,0.37,1.00,0.19,0.00,0.00,0.17,0.01,0.00,0.31,0.01,0.62,0.87,0.55,0.52,0.52,1.00,1.00,1.00,0.71
DER_deltar_tau_lep,-0.00,0.23,0.04,0.58,-0.54,-0.30,-0.30,-0.30,1.00,-0.15,-0.43,0.05,-0.21,-0.30,-0.20,0.00,-0.01,-0.07,0.00,-0.00,-0.40,-0.00,-0.41,-0.35,-0.34,-0.30,-0.30,-0.30,-0.30,-0.30,-0.45
DER_pt_tot,-0.00,0.05,0.02,-0.00,0.31,0.27,0.25,0.27,-0.15,1.00,0.38,0.04,0.18,0.27,0.10,0.00,0.00,0.11,0.01,-0.00,0.27,0.00,0.45,0.36,0.20,0.19,0.19,0.28,0.27,0.27,0.40


Having a look at the table below, we find that there are some features that have a correlation bigger than 0.9. We can considere this values as an obvious relationship between the features and so we will remove then.

In [10]:
# Calculate absolute of all corr values
corr_matrix = corr_matrix.abs()

# Get upper triangle of correlation matrix
mask = np.triu(np.ones(corr_matrix.shape), k=1)
upperTriangle = corr_matrix.where(mask.astype(bool))

# Find index of features with corr > 0.9
del_features = [events.columns.get_loc(column) - 2 for column in upperTriangle.columns if any(upperTriangle[column] > 0.9)]
print('Index of features to delete:')
del_features

Index of features to delete:


[5, 6, 12, 21, 24, 25, 26, 27, 28, 29]

In [11]:
def remove_related_features(tX):
    """Remove related features obtained during the exporation of the data."""
    del_features = [5, 6, 12, 21, 24, 25, 26, 27, 28, 29]
    np.delete(tX, del_features, 1)
    
    return tX

## Generate predictions and save ouput in csv format for submission:

Generate submission csv file.

In [12]:
my_file = Path("../data/test.csv")
if not my_file.is_file():
    with zipfile.ZipFile('../data/test.csv.zip', 'r') as zip_ref:
        zip_ref.extractall('../data')

DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

Split data and process.

In [13]:
masks_train = get_masks(tX)
masks_test = get_masks(tX_test)

tX = remove_related_features(tX)
tX_test = remove_related_features(tX_test)

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)

Get best method to fit our model:
- **Least Squares**

In [14]:
weights_LS = []
total_loss = 0

for idx in range(len(masks_train)):
    weight, loss = least_squares(y_train, x_train)
    
    weights_LS.append(weight)
    total_loss += loss

print('Loss with Least Squares: ', total_loss)

Loss with Least Squares:  1.0861805645920697


- **Gradient descent**

In [15]:
initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 1e-10

weights_GD = []
total_loss = 0

for idx in range(len(masks_train)):
    weight, loss = least_squares_GD(y_train, x_train, initial_w, max_iters, gamma)
    
    weights_GD.append(weight)
    total_loss += loss

print('Loss with Gradient descent: ', total_loss)

Loss with Gradient descent:  1.4998328998356019


- **Stochastic Gradient descent**

In [16]:
initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 1e-10

weights_SGD = []
total_loss = 0

for idx in range(len(masks_train)):
    weight, loss = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)
    
    weights_SGD.append(weight)
    total_loss += loss

print('Loss with Stochastic Gradient descent: ', total_loss)

Loss with Stochastic Gradient descent:  1.4999605871528854


- **Ridge regression**

In [17]:
lambdas = [0.002, 0.001, 0.001]

weights_RR = []
total_loss = 0

for idx in range(len(masks_train)):
    weight, loss = ridge_regression(y, tX, lambdas[idx])
    
    weights_RR.append(weight)
    total_loss += loss

print('Loss with Ridge regression: ', total_loss)

Loss with Ridge regression:  1.0190723996890978


- **Logistic regression**

In [18]:
initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 1e-30

weights_LR = []
total_loss = 0

for idx in range(len(masks_train)):
    weight, _ = logistic_regression(y_train, x_train, initial_w, max_iters, gamma)
    loss = compute_loss(y_train, x_train, weight)
    
    weights_LR.append(weight)
    total_loss += loss

print('Loss with Logistic regression: ', total_loss)

Loss with Logistic regression:  1.499999999999971


- **Regularized Logistic Regression**

In [19]:
initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 1e-30
lambda_ = 1e-10

weights_LRL = []
total_loss = 0

for idx in range(len(masks_train)):
    weight, _ = reg_logistic_regression(y_train, x_train, lambda_,initial_w, max_iters, gamma)
    loss = compute_loss(y_train, x_train, weight)
    
    weights_LRL.append(weight)
    total_loss += loss

print('Loss with Regularized Logistic Regression: ', total_loss)

Loss with Regularized Logistic Regression:  1.499999999999971


Get predictions with best method.

In [20]:
# Vector in which we will store the predictions
y_pred = np.zeros(tX_test.shape[0])

for idx in range(len(masks_test)):
    x_test = tX_test[masks_test[idx]]
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_test = replace_na_values(x_test)

    y_test_pred = predict_labels(weights_RR[idx], x_test)
    y_pred[masks_test[idx]] = y_test_pred

## Cross validation
Run cross validation to compute the accurancy of the different methods.

In [21]:
seed = 10

- **Least squares**

In [22]:
accurancies = []

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)
    
    # Split data in k-fold
    k_fold = 10
    k_indices = build_k_indices(y_train, k_fold, seed)


    for k in range(k_fold):
        accurancy = cross_validation(y_train, x_train, k_indices, k, least_squares_SGD, initial_w=initial_w, max_iters=max_iters, gamma=gamma)
        accurancies.append(accurancy)


print("Accurancy")
print("Average: %f" % np.mean(accurancies))
print("Standar deviation: %f" % np.std(accurancies))
print("Min: %f" % np.min(accurancies))
print("Max: %f" % np.max(accurancies))

Accurancy
Average: 0.662481
Standar deviation: 0.074804
Min: 0.448580
Max: 0.753979


- **Gradient Descent**

In [23]:
# Model parameters
initial_w = np.zeros(tX.shape[1])
gamma = 1e-10
max_iters = 50

accurancies = []

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)
    
    # Split data in k-fold
    k_fold = 10
    k_indices = build_k_indices(y_train, k_fold, seed)


    for k in range(k_fold):
        accurancy = cross_validation(y_train, x_train, k_indices, k, least_squares_GD, initial_w=initial_w, max_iters=max_iters, gamma=gamma)
        accurancies.append(accurancy)


print("Accurancy")
print("Average: %f" % np.mean(accurancies))
print("Standar deviation: %f" % np.std(accurancies))
print("Min: %f" % np.min(accurancies))
print("Max: %f" % np.max(accurancies))

Accurancy
Average: 0.687682
Standar deviation: 0.042842
Min: 0.634640
Max: 0.753979


- **Stochastic Gradient Descent**

In [24]:
# Model parameters
initial_w = np.zeros(tX.shape[1])
gamma = 1e-10
max_iters = 50

accurancies = []

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)
    
    # Split data in k-fold
    k_fold = 10
    k_indices = build_k_indices(y_train, k_fold, seed)


    for k in range(k_fold):
        accurancy = cross_validation(y_train, x_train, k_indices, k, least_squares_SGD, initial_w=initial_w, max_iters=max_iters, gamma=gamma)
        accurancies.append(accurancy)


print("Accurancy")
print("Average: %f" % np.mean(accurancies))
print("Standar deviation: %f" % np.std(accurancies))
print("Min: %f" % np.min(accurancies))
print("Max: %f" % np.max(accurancies))

Accurancy
Average: 0.662499
Standar deviation: 0.074803
Min: 0.448580
Max: 0.753979


- **Ridge regression**

In [25]:
# Model parameters
lambdas = [0.002, 0.001, 0.001]

accurancies = []

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)
    
    # Split data in k-fold
    k_fold = 10
    k_indices = build_k_indices(y_train, k_fold, seed)


    for k in range(k_fold):
        accurancy = cross_validation(y_train, x_train, k_indices, k, ridge_regression, lambda_=lambdas[idx])
        accurancies.append(accurancy)

print("Accurancy")
print("Average: %f" % np.mean(accurancies))
print("Standar deviation: %f" % np.std(accurancies))
print("Min: %f" % np.min(accurancies))
print("Max: %f" % np.max(accurancies))

Accurancy
Average: 0.750645
Standar deviation: 0.042721
Min: 0.702992
Max: 0.818937


- **Logistic regression**

In [26]:
# Model parameters
initial_w = np.zeros(tX.shape[1])
gamma = 1e-30
max_iters = 50

accurancies = []

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)
    
    # Split data in k-fold
    k_fold = 10
    k_indices = build_k_indices(y_train, k_fold, seed)


    for k in range(k_fold):
        accurancy = cross_validation(y_train, x_train, k_indices, k, logistic_regression, initial_w=initial_w, max_iters=max_iters, gamma=gamma)
        accurancies.append(accurancy)


print("Accurancy")
print("Average: %f" % np.mean(accurancies))
print("Standar deviation: %f" % np.std(accurancies))
print("Min: %f" % np.min(accurancies))
print("Max: %f" % np.max(accurancies))

Accurancy
Average: 0.646651
Standar deviation: 0.078735
Min: 0.547284
Max: 0.753979


- **Regularized logistic regression**

In [27]:
# Model parameters
initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 1e-30
lambda_ = 1e-10

accurancies = []

for idx in range(len(masks_train)):
    x_train = tX[masks_train[idx]]
    y_train = y[masks_train[idx]]
    
    # We expect to see this RuntimeWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        x_train = replace_na_values(x_train)
    
    # Split data in k-fold
    k_fold = 10
    k_indices = build_k_indices(y_train, k_fold, seed)


    for k in range(k_fold):
        accurancy = cross_validation(y_train, x_train, k_indices, k, reg_logistic_regression, lambda_=lambda_, initial_w=initial_w, max_iters=max_iters, gamma=gamma)
        accurancies.append(accurancy)

print("Accurancy")
print("Average: %f" % np.mean(accurancies))
print("Standar deviation: %f" % np.std(accurancies))
print("Min: %f" % np.min(accurancies))
print("Max: %f" % np.max(accurancies))

Accurancy
Average: 0.646651
Standar deviation: 0.078735
Min: 0.547284
Max: 0.753979


Generate predictions file.

In [28]:
OUTPUT_PATH = '../data/submission.csv'
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)